In [1]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [2]:
myteam = 'Team Whiting'

In [3]:
driver = webdriver.Firefox()
driver.get(r'https://www.espn.com')
# driver.get(r"https://fantasy.espn.com/football/draft?leagueId=97522804&seasonId=2020&teamId=1&memberId={70782481-065A-4843-8A38-A138B0BADDA1}")

In [4]:
df = pd.read_pickle('players.pkl')

In [5]:
print(driver.title)
print(driver.current_url)

Fantasy Football Draft - ESPN
https://fantasy.espn.com/football/draft?leagueId=80338282&seasonId=2020&teamId=7&memberId={70782481-065A-4843-8A38-A138B0BADDA1}


# Wait until the draft room opens

In [6]:
def update_rosters():
    teams = driver.find_elements_by_xpath("/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[1]/div[1]/div[2]/div[1]/div/select/option")
    rosters = {}
    team_map = {}
    for t in teams:
        t.click()
        tnum = int(t.get_property('value'))
        team_ele = driver.find_elements_by_xpath("/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[1]/div[1]/div[2]/div[2]/div/section/div/div/div[2]/table/tbody/tr")
        team = pd.DataFrame([r.text.splitlines() for r in team_ele])[[0,1]].rename(columns={0:'position',1:'player'})
        team['player'] = team['player'].replace('Empty', np.nan)
        rosters[tnum] = team
        team_map[tnum] = t.text
    return rosters, team_map

In [7]:
rosters, team = update_rosters()

In [16]:
MYTEAM = list(team.keys())[list(team.values()).index('TEAM WHITING')]

In [17]:
open_spots = rosters[MYTEAM]['player'].isna()

In [18]:
rosters[MYTEAM][open_spots]['position'].value_counts()

BE      7
WR      2
RB      2
FLEX    1
QB      1
K       1
D/ST    1
TE      1
Name: position, dtype: int64

In [24]:
ROUNDS = rosters[MYTEAM][open_spots]['position'].value_counts().sum()

In [19]:
TEAMS = len(team)

In [20]:
def pick_history():
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[2]/div[2]/div/nav/ul/li[2]/button').click()
    
def avail_players():
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[2]/div[2]/div/nav/ul/li[1]/button').click()

In [25]:
rpick_id = lambda PICK,r,p: f"/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[2]/div[2]/div/div/div[2]/div/div[2]/div{f'[{r}]' if PICK > TEAMS else ''}/div[2]/div/div[1]/div/div/div[3]/div[{p}]/div/div/div[2]/div/div[2]/div/div/div/div/div/div[1]/img[1]"
rpick_name = lambda PICK,r,p: f"/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[2]/div[2]/div/div/div[2]/div/div[2]/div{f'[{r}]' if PICK > TEAMS else ''}/div[2]/div/div[1]/div/div/div[3]/div[{p}]/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div[1]/span/span/a"

do_pick_id = lambda r,p: f"/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[2]/div[2]/div/div/div[2]/div/div[2]/div[{r}]/div[2]/div/div[1]/div/div/div[3]/div[{p}]/div/div/div[2]/div/div[2]/div/div/div/div/div/div[1]/img[1]"
do_pick_name = lambda r,p: f"/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[3]/div[2]/div[2]/div/div/div[2]/div/div[2]/div[{r}]/div[2]/div/div[1]/div/div/div[3]/div[{p}]/div/div/div[2]/div/div[2]/div/div/div/div/div/div[2]/div[1]/span/span/a"


def scrape_pickids(PICK, r,p):
    if PICK != TEAMS * ROUNDS:
        string = rpick_id(PICK,r,p)
        ns = rpick_name(PICK,r,p)
    else:
        string = do_pick_id(r,p)
        ns = do_pick_name(r,p)
    
    element = driver.find_element_by_xpath(string)
    imgref = element.get_property('src')
    beg = imgref.find('full') + 5
    end = imgref.find('.png')
    
    if beg == 4:
        beg = imgref.find('nfl/500/') + 8
        pid = imgref[beg:end]
    else:
        pid = int(imgref[beg:end])
    
    ne = driver.find_element_by_xpath(ns)
    name = ne.text
    return r,p,pid,name

def get_current_pick(exception=TEAMS*ROUNDS):
    try:
        cpe = driver.find_element_by_xpath('/html/body/div[1]/div[1]/section/div/div[2]/main/div/div/div[2]/div/div[2]/div/div[2]')
        picktxt = cpe.text
        beg = picktxt.find('PICK') + 5
        current_pick = int(picktxt[beg:]) - 1 
    except:
        current_pick = exception
    return current_pick

def update():
    current_pick = get_current_pick()
    pick_history()
#     current_pick = 128
    teams = TEAMS
    cR = (current_pick-1) // teams + 1
    cP = (current_pick-1) % teams + 1
    picks_list = [(r,p) for r in range(1,cR) for p in range(1,teams+1)] # FIX THIS
    picks_list = picks_list + [(cR, p) for p in range(1, cP+1)]
    pickids = [scrape_pickids(current_pick,r,p) for r,p in picks_list]
    print(len(pickids))
    avail_players()
    return pickids

In [26]:
get_current_pick()

2

In [29]:
pickids = update()

3


In [30]:
pickids

[(1, 1, 3117251, 'Christian McCaffrey'),
 (1, 2, 2976316, 'Michael Thomas'),
 (1, 3, 3929630, 'Saquon Barkley')]

In [31]:
def process_update():
    dfpids = pd.DataFrame(update())
    dfpids.columns = ['round','pick','espn_id','espn_name']
    dfpids = dfpids.set_index('espn_id')
    return dfpids

In [32]:
dfp = process_update()

3


In [33]:
dfp

,round,pick,espn_name
espn_id,,,
3117251,1,1,Christian McCaffrey
2976316,1,2,Michael Thomas
3929630,1,3,Saquon Barkley


In [34]:
dfp.index

Int64Index([3117251, 2976316, 3929630], dtype='int64', name='espn_id')

In [35]:
df['espn_id'].isin(dfp.index)

0      True
1      True
2     False
3     False
4     False
      ...  
27    False
28    False
29    False
30    False
31    False
Name: espn_id, Length: 270, dtype: bool

In [36]:
df['picked'] = df['espn_id'].isin(dfp.index)

In [37]:
df['picked'] = df['espn_id'].isin(dfp.index)
df['round'] = df['espn_id'].map(dfp['round'])
df['rpick'] = df['espn_id'].map(dfp['pick'])

In [38]:
def filter_picks(df):
    dfp = process_update()
    df['picked'] = df['espn_id'].isin(dfp.index)
    df['round'] = df['espn_id'].map(dfp['round'])
    df['rpick'] = df['espn_id'].map(dfp['pick'])
    return df

In [39]:
df1 = filter_picks(df)

3


In [42]:
df1.head(60)

,playerId,player,team,position,age,exp,bye,points,lower,upper,...,adp,auctionValue,adpDiff,cost,ppdollar,espn_names,espn_id,picked,round,rpick
0,2557997.0,Christian McCaffrey,CAR,RB,24.0,3.0,NaN,288.506696,265.133189,302.219263,...,1.9533,72.680,-0.9533,80.0,3.606334,Christian McCaffrey,3117251,True,1.0,1.0
1,2560968.0,Saquon Barkley,NYG,RB,23.0,2.0,NaN,270.049352,244.501192,297.133185,...,2.0533,67.725,-0.0533,75.0,3.600658,Saquon Barkley,3929630,True,1.0,3.0
2,2555224.0,Ezekiel Elliott,DAL,RB,25.0,4.0,NaN,256.383828,238.539195,278.376832,...,3.5133,60.345,-0.5133,67.0,3.826624,Ezekiel Elliott,3051392,False,NaN,NaN
3,2556075.0,Derrick Henry,TEN,RB,26.0,4.0,NaN,237.677703,217.315797,246.640252,...,16.5267,50.265,-12.5267,56.0,4.244245,Derrick Henry,3043078,False,NaN,NaN
4,2557991.0,Dalvin Cook,MIN,RB,25.0,3.0,NaN,231.766350,192.707601,247.833873,...,7.7267,54.665,-2.7267,61.0,3.799448,Dalvin Cook,3116593,False,NaN,NaN
5,2558019.0,Alvin Kamara,NO,RB,24.0,3.0,NaN,218.905602,196.300000,238.737475,...,4.0400,57.420,1.9600,64.0,3.420400,Alvin Kamara,3054850,False,NaN,NaN
6,2562377.0,Josh Jacobs,LVR,RB,50.0,1.0,NaN,211.845761,186.777198,236.761718,...,20.5667,41.305,-13.5667,46.0,4.605343,Josh Jacobs,4047365,False,NaN,NaN
7,2558116.0,Aaron Jones,GB,RB,26.0,3.0,NaN,211.514392,186.400000,230.358581,...,20.7067,39.620,-12.7067,44.0,4.807145,Aaron Jones,3042519,False,NaN,NaN
8,2562722.0,Miles Sanders,PHI,RB,50.0,1.0,NaN,211.339254,190.608793,226.553109,...,25.9967,40.615,-16.9967,45.0,4.696428,Miles Sanders,4045163,False,NaN,NaN
9,2557976.0,Joe Mixon,CIN,RB,24.0,3.0,NaN,211.122397,189.800000,229.531991,...,12.6767,46.190,-2.6767,51.0,4.139655,Joe Mixon,3116385,False,NaN,NaN


In [ ]:
dfp = process_update()

In [ ]:
df['espn_id'].map(dfp['round'])

In [ ]:
df['picked'] = df['espn_id'].isin(dfpids['espn_id'])

In [ ]:
df[~df['picked']]

In [ ]:
pickids = update()

In [ ]:
driver.close()

In [ ]:
df['picked'] = df['espn_id'].isin(dfpids['espn_id'])

In [ ]:
df[~df.picked]